This is a Juypter notebook template that can be used to visualize the groundwater level measurements from one of the 10 PVACD monitoring wells. THe first cell defines a simple python function called `fetch`. This function is used for all of the API queries.  Because we are working with web APIs is is recommended to use `async` functions.

In [1]:
import pyodide
import matplotlib.pyplot as plt

async def fetch(path, *args, **kw):
    url = f'https://st2.newmexicowaterdata.org/FROST-Server/v1.1/{path}'
    a = await pyodide.http.pyfetch(url=url,
                                 method="get",
                                 )
    data = await a.json()
    return data

Configure the next cell with the `location_name` of the well you would like to plot. e.g `Orchard Park`. The cell will then `fetch` the **Location,Thing and Datastream** from `ST2`

In [ ]:
location_name = 'Orchard Park'
resp = await fetch(f"Locations?$filter=startswith(name, '{location_name}')&$expand=Things/Datastreams")
location = resp['value'][0]
ds_id = location['Things'][0]['Datastreams'][0]['@iot.id']

Now that we have the Groundwater Levels Datastream id with can retrieve the Observations associated with it

In [2]:
resp = await fetch(f'Datastreams({ds_id})/Observations?$orderby=phenomenonTime desc')
data = resp['value']

Use `matplotlib.pyplot` to plot a timeseries of Depth To Water

In [ ]:
ys = [d['result'] for d in data]
xs = [d['phenomenonTime'] for d in data]
plt.plot(xs,ys)
plt.gca().invert_yaxis()
plt.gca().invert_xaxis()

plt.title(f"{location_name}")
plt.xlabel("Time")
plt.ylabel("DepthToWater (bgs ft)")
xticks = [xi for (i,xi) in enumerate(xs) if not i%100]
xlabels = [xi.split('T')[0] for xi in xticks]
plt.xticks(xticks, xlabels,
           rotation=45)
plt.tight_layout()
plt.show()